In [11]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 9.8 MB/s eta 0:00:00


In [18]:
from google.colab import drive
import os
import warnings
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import contractions
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_short, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, stem_text
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

warnings.filterwarnings('ignore')
# This will prompt for authorization.
drive.mount('/content/drive')

# Change to assignment directory
path_to_folder = '/content/drive/My Drive/slef'
os.chdir(path_to_folder) 


[nltk_data] Downloading package wordnet to /root/nltk_data...


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Read HTML page using requests
url = 'https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html'
html = requests.get(url,verify=False).content

# Parse HTML using Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
dom =  etree.HTML(str(soup))
last_stmt_link = dom.xpath('//a[contains(text(), "Last Statement")]')

output = []

for elem in last_stmt_link:
    try:
        link = elem.get('href')
        # Read HTML page for current prisoner's last statement
        html = requests.get(f'https://www.tdcj.texas.gov/death_row/{link}',verify=False).content
        soup = BeautifulSoup(html, 'html.parser')
        # Get prisoner's last statement text
        text = soup.select('p:nth-child(11)')[0].get_text().strip()
        # Assign 'None' if prisoner said nothing
        if text != '':
            output.append(text)
    except:
        print(f"page {link} not found")
        text = 'None'

In [ ]:
statement_df = pd.DataFrame(output, columns=['Last Statement'])

In [ ]:
statement_df.to_csv('data.csv')

In [7]:
df = pd.read_csv('data.csv')

In [8]:
df=df[df['Last Statement'] != 'No last statement given.']

In [9]:
df.head()
df.shape

,Unnamed: 0,Last Statement
0,0,What is occurring here tonight is not justice...
1,1,"Vetta, Jared, Ray I’m sorry, no I’m not sorry..."
2,2,"Yes ma’am, I want to thank y’all. I love y’al..."
3,3,I would like to apologize to the Nix family f...
5,5,I want to take this moment to be shared with e...


In [14]:
def preprocess(doc):
  

# Contraction expander
def remove_contractions(doc):
  return ' '.join([contractions.fix(word) for word in doc.split()]) # Expands contractions from text

# Lemmatization
py_lemmatizer = WordNetLemmatizer()
def lemmatize(doc):
  return [py_lemmatizer.lemmatize(word) for word in doc]


df['Clean']=df['Last Statement'].apply(lambda doc: remove_contractions(doc))

In [15]:
# Filtering
FILTERS=[lambda x: x.lower(), strip_tags, strip_punctuation,
    strip_multiple_whitespaces, strip_numeric,
    remove_stopwords, strip_short]
df['Tokenized'] = df['Clean'].apply(lambda x: preprocess_string(x, FILTERS)) # Removes tags, punctuation, whitespaces, numerics and stopwords.

In [19]:
df['Tokenized'] = df['Tokenized'].apply(lambda doc: lemmatize(doc)) # Lemmatizes

In [20]:
print("------ Before Text Preprocessing ------\n")
print(df['Last Statement'][9])
print('')
print("------ After Text Preprocessing ------\n")
print(df['Tokenized'][9])

------ Before Text Preprocessing ------

Ok to start with I would like to thank everyone that has stayed;  Karen, Linda, Barry, Danny, Barbara, God bless each and every one of you. I  have a message to the Irby family. The shootout occurred June 27, 1990. One  week later a police officer on his day off named Michael Garret showed up to my  cell. He was wearing civilian clothes. I thought he was a chaplain. He came to the  back where it is super seg. The guys there began to ridicule him because the  officer brought me a small Bible.

------ After Text Preprocessing ------

['start', 'like', 'thank', 'stayed', 'karen', 'linda', 'barry', 'danny', 'barbara', 'god', 'bless', 'message', 'irby', 'family', 'shootout', 'occurred', 'june', 'week', 'later', 'police', 'officer', 'day', 'named', 'michael', 'garret', 'showed', 'cell', 'wearing', 'civilian', 'clothes', 'thought', 'chaplain', 'came', 'super', 'seg', 'guy', 'began', 'ridicule', 'officer', 'brought', 'small', 'bible']
